In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [83]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense

In [84]:
bike_train = pd.read_excel('./bike_train.xlsx')
bike_test = pd.read_excel('./bike_test.xlsx')

In [85]:
bike_train.drop(columns=['dteday', 'instant', 'casual', 'registered'], axis=1, inplace=True)

In [86]:
bike_test.drop(columns=['dteday', 'instant'], axis=1, inplace=True)

In [87]:
bike_train.head()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,1


In [88]:
0.75 * bike_train.shape[0]

8999.25

In [89]:
training_set = bike_train[:9000].iloc[:, 12:13]
test_set = bike_train[9000:].iloc[:, 12:13]

In [90]:
sc = MinMaxScaler(feature_range=(0, 1))
scale_training = sc.fit_transform(training_set)

In [91]:
X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(scale_training[i-60:i, 0])
    y_train.append(scale_training[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [92]:
model = Sequential()
model.add(LSTM(units=50,
        return_sequences=True,input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X_train,y_train,epochs=20,batch_size=32)

Epoch 1/20
62/62 [==============================] - 50s 203ms/step - loss: 0.0116
Epoch 2/20
62/62 [==============================] - 12s 188ms/step - loss: 0.0088
Epoch 3/20
62/62 [==============================] - 11s 184ms/step - loss: 0.0074
Epoch 4/20
62/62 [==============================] - 13s 206ms/step - loss: 0.0073
Epoch 5/20
62/62 [==============================] - 13s 204ms/step - loss: 0.0073
Epoch 6/20
62/62 [==============================] - 15s 241ms/step - loss: 0.0071
Epoch 7/20
62/62 [==============================] - 12s 200ms/step - loss: 0.0071
Epoch 8/20
62/62 [==============================] - 15s 235ms/step - loss: 0.0069
Epoch 9/20
62/62 [==============================] - 13s 209ms/step - loss: 0.0066
Epoch 10/20
62/62 [==============================] - 12s 193ms/step - loss: 0.0065
Epoch 11/20
62/62 [==============================] - 12s 192ms/step - loss: 0.0065
Epoch 12/20
62/62 [==============================] - 12s 191ms/step - loss: 0.0063
Epoch 13/20
6

In [101]:
dataset_total = pd.concat((bike_train['cnt'], test_set['cnt']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(bike_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_cnt = model.predict(X_test)
predicted_cnt = sc.inverse_transform(predicted_cnt)

c:\Users\cajas\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


169/169 [==============================] - 8s 48ms/step


In [128]:
df = pd.DataFrame(predicted_cnt)
df.rename(columns={0: 'pred'}, inplace=True)

In [129]:
df.to_csv('./rupertsky.csv', index=False, header=True)